In [6]:
import news

In [7]:
news.GoogleNews().get_titles()

['What Makes Alphabet (GOOG) a Good Long-Term Investment? - Yahoo Finance',
 'Why do Long-Term Investors Love Alphabet (GOOG)? - Yahoo Finance',
 'BIDU vs. GOOG: Which Stock Should Value Investors Buy Now? - Yahoo Finance',
 'My Top Stock to Buy in a New Bull Market - The Motley Fool',
 'After Meta, Google Plans To Cut Off Canada From News - Alphabet (NASDAQ:GOOG), Alphabet (NASDAQ:GOOGL), M - Benzinga',
 '3 No-Brainer Stocks to Buy Right Now for Less Than $200 - The Motley Fool',
 'Google: Why I See New Highs Coming (Rating Upgrade) (NASDAQ:GOOG) - Seeking Alpha',
 '2 Cheap AI Stocks to Buy Right Now - The Motley Fool',
 'Google: Another Downgrade Hypes The Threat (NASDAQ:GOOG) - Seeking Alpha',
 'Google parent Alphabet’s stock hit with two downgrades this week - Yahoo Finance',
 'Can Alphabet Inc Class C (GOOG) Stock Rise to the Top of Communication Services Sector Friday? - InvestorsObserver',
 'Does Alphabet Inc Class C (GOOG) Have What it Takes to be in Your Portfolio Thursday? - 